In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from segmentation import *
from model import *

In [2]:
model.load_weights('actions.h5')

In [3]:
from sklearn.metrics import multilabel_confusion_matrix,accuracy_score

yhat = model.predict(X_train)

ytrue = np.argmax(y_train,axis=1).tolist()
yhat = np.argmax(yhat,axis=1).tolist()


In [4]:
multilabel_confusion_matrix(ytrue,yhat)

array([[[56,  2],
        [ 1, 26]],

       [[56,  0],
        [ 0, 29]],

       [[55,  1],
        [ 2, 27]]], dtype=int64)

In [5]:
accuracy_score(ytrue,yhat)

0.9647058823529412

In [6]:
DATA_PATH = 'E:\\Final Year Project\\Express-U\data\\vid_dataset'

actions = np.array(['hello','thanks','iloveyou'])

no_of_sequences = 30

sequence_length = 30

In [ ]:
label_map = {label:num for num,label in enumerate(actions)}
sequences,labels = [],[]
for action in actions:
  for sequence in range(no_of_sequences):
    window = []
    for frame_num in range(sequence_length):
      res = np.load(os.path.join(DATA_PATH,action,str(sequence),f'{frame_num}.npy'))
      window.append(res)
    sequences.append(window)
    labels.append(label_map[action])

In [7]:
sequence = []
sentence = []
predictions = []
threshold = 0.8
colors = [(245,117,16),(117,245,16),(16,117,245)]

cap = cv2.VideoCapture(0)

with mp_holistic.Holistic(min_detection_confidence=0.5,min_tracking_confidence=0.5) as holistic:
  while cap.isOpened():
    ret,frame  = cap.read()
    # frame = cv2.resize(frame,(800,600))

    image,results = mediapipe_detection(frame,holistic)

    draw_styled_landmarks(image,results)

    keypoints = extract_keypoints(results)
    sequence.append(keypoints)
    sequence = sequence[-30:]

    if len(sequence) == 30:
      res = model.predict(np.expand_dims(sequence,axis=0))[0]
      # print(actions[np.argmax(res)])
      predictions.append(np.argmax(res))

      if np.unique(predictions[-10:])[0] == np.argmax(res):
        if res[np.argmax(res)] > threshold:

          if len(sentence) > 0:
            if actions[np.argmax(res)] != sentence[-1]:
              sentence.append(actions[np.argmax(res)])
          else:
              sentence.append(actions[np.argmax(res)])
      
      if len(sentence) > 5:
        sentence = sentence[-5:]
      
      image = prob_viz(res,actions,image,colors)

    cv2.rectangle(image,(0,0),(640,40),(245,117,16),-1)
    cv2.putText(image," ".join(sentence),(3,25),cv2.FONT_HERSHEY_SIMPLEX,1,(255,255,255),2,cv2.LINE_AA)

    cv2.imshow('result',image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
      break
  cap.release()
  cv2.destroyAllWindows()

In [8]:
sentence

['hello', 'iloveyou', 'thanks', 'iloveyou', 'thanks']

In [9]:
np.unique(predictions[-10:])[0] == np.argmax(res)

False